In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# split train data into train set and valid set
train_images, valid_images, train_labels, valid_labels = train_test_split(
  np.load('train_images.npy'),
  np.load('train_labels.npy'), 
  test_size = 0.2,
  random_state = 42,
)

test_images = np.load('test_images.npy')
test_labels = np.load('test_labels.npy')

In [ ]:
train_images.shape, train_labels.shape, valid_images.shape, valid_labels.shape, test_images.shape, test_labels.shape

In [ ]:
class MomentumOptimizer:
    def __init__(self, parameters, lr=0.01, momentum=0.9):
        self.parameters = parameters
        self.lr = lr
        self.momentum = momentum
        self.v = [np.zeros_like(p) for p in parameters]
    
    def update(self, grads):
        for i, (param, grad) in enumerate(zip(self.parameters, grads)):
            self.v[i] = self.momentum * self.v[i] - self.lr * grad
            param += self.v[i]

In [ ]:
class AdamOptimizer:
    def __init__(self, parameters, lr=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.parameters = parameters
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = [np.zeros_like(p) for p in parameters]
        self.v = [np.zeros_like(p) for p in parameters]
        self.t = 0
    
    def update(self, grads):
        self.t += 1
        lr_t = self.lr * np.sqrt(1 - self.beta2 ** self.t) / (1 - self.beta1 ** self.t)

        for i, (param, grad) in enumerate(zip(self.parameters, grads)):
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * grad
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * (grad ** 2)
            param -= lr_t * self.m[i] / (np.sqrt(self.v[i]) + self.epsilon)

In [ ]:
# Define the network
class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, optimizer=None):
        # Initialize weights and biases
        #self.w1 = np.random.randn(input_size, hidden_size) * 0.01
        self.w1 = np.random.randn(input_size, hidden_size) * np.sqrt(2. / input_size)
        self.b1 = np.zeros((1, hidden_size))

        #self.w2 = np.random.randn(hidden_size, hidden_size) * 0.01
        self.w2 = np.random.randn(hidden_size, hidden_size) * np.sqrt(2. / hidden_size)
        self.b2 = np.zeros((1, hidden_size))

        #self.w3 = np.random.randn(hidden_size, output_size) * 0.01
        self.w3 = np.random.randn(hidden_size, output_size) * np.sqrt(2. / hidden_size)
        self.b3 = np.zeros((1, output_size))    

        self.optimizer = optimizer

    # z: (m, hidden_size)
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    # z: (m, hidden_size)
    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    def relu(self, z):
        return np.maximum(0, z)

    def relu_derivative(self, z):
        return (z > 0).astype(float)


    # compute the cross-entropy loss
    # y_true: true labels     (m, output_size)
    # y_pred: predicted labels (m, output_size)
    def compute_loss(self, y_true, y_pred):
        m = y_true.shape[0]
        loss = -np.sum(y_true * np.log(y_pred + 1e-8)) / m
        return loss
    
    # X: input data (m, input_size)
    def forward_propagation(self, X):
        # z1: X * w1 + b1 (m, hidden_size) 
        self.z1 = np.dot(X, self.w1) + self.b1
        self.a1 = self.relu(self.z1)

        # z2: a1 * w2 + b2 (m, hidden_size)
        self.z2 = np.dot(self.a1, self.w2) + self.b2
        self.a2 = self.relu(self.z2)

        # z3: a2 * w2 + b3 (m, output_size)
        self.z3 = np.dot(self.a2, self.w3) + self.b3
        self.a3 = self.softmax(self.z3)
        return self.a3
    
    # X: input data  (m, input_size)
    # Y: true labels (m, output_size)
    def backward_propagation(self, X, y):
        m = X.shape[0]

        dz3 = self.a3 - y
        dw3 = np.dot(self.a2.T, dz3) / m
        db3 = np.sum(dz3, axis=0, keepdims=True) / m

        
        #dz2 = np.dot(dz3, self.w3.T) * self.a2 * (1 - self.a2)
        dz2 = np.dot(dz3, self.w3.T) * self.relu_derivative(self.z2)
        dw2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        #dz1 = np.dot(dz2, self.w2.T) * self.a1 * (1 - self.a1)
        dz1 = np.dot(dz2, self.w2.T) * self.relu_derivative(self.z1)
        dw1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        return dw1, db1, dw2, db2, dw3, db3
    
    
    def update_parameters(self, grads, learning_rate=0.01):
        if self.optimizer is None:
            dw1, db1, dw2, db2, dw3, db3 = grads
            self.w1 -= learning_rate * dw1
            self.b1 -= learning_rate * db1
            self.w2 -= learning_rate * dw2
            self.b2 -= learning_rate * db2
            self.w3 -= learning_rate * dw3
            self.b3 -= learning_rate * db3
        else:
            self.optimizer.update(grads)

In [ ]:
# Parameters
input_size = 784    # 28x28 pixels
hidden_size = 128   # Experiment with different sizes
output_size = 10    # Digits 0-9

# Initialize the neural network
nn = SimpleNeuralNetwork(input_size, hidden_size, output_size)

#optimizer = MomentumOptimizer([nn.w1, nn.b1, nn.w2, nn.b2, nn.w3, nn.b3])
#optimizer = AdamOptimizer([nn.w1, nn.b1, nn.w2, nn.b2, nn.w3, nn.b3])
#nn.optimizer = optimizer

In [ ]:
# Train the network
def train(model, X_train, y_train, X_val, y_val, X_test, y_test, epochs = 30, learning_rate = 0.01, batch_size = 128, patience = 5, min_delta = 1e-6):
    loss_train_history = []
    loss_val_history = []
    test_accuracy_history = []

    # early stopping variables
    #patience_count = 0
    #best_val_loss = np.inf

    step = 0
    
    for epoch in range(epochs):
        for i in range(0, X_train.shape[0], batch_size):
            X_train_batch = X_train[i:i+batch_size]
            y_train_batch = y_train[i:i+batch_size]
            
            # Forward propagation
            y_train_pred = model.forward_propagation(X_train_batch)
            
            # Compute loss
            loss_train = model.compute_loss(y_train_batch, y_train_pred)
            loss_train_history.append(loss_train)
            
            # Backward propagation
            grads = model.backward_propagation(X_train_batch, y_train_batch)
            
            # Update parameters
            model.update_parameters(grads, learning_rate)
        
            # Validation loss
            y_pred_val = model.forward_propagation(X_val)
            loss_val = model.compute_loss(y_val, y_pred_val)
            loss_val_history.append(loss_val)

            step += 1

            # test accuracy
            y_test_pred = model.forward_propagation(X_test)
            test_accuracy = np.sum(np.argmax(y_test_pred, axis=1) == np.argmax(y_test, axis=1)) * 1.0 / y_test.shape[0]
            test_accuracy_history.append(test_accuracy)

            print(f"Epoch {epoch}, Step {step}, Train Loss: {loss_train}, Validation Loss: {loss_val}, Test Set Accuracy: {test_accuracy}")

            # early stopping
            #if best_val_loss - loss_val > min_delta:
            #    best_val_loss = loss_val
            #    patience_count = 0
            #else:
            #    patience_count += 1

            #if patience_count == patience:
            #    print(f'Early stopping at epoch {epoch}, step {step}, val loss {loss_val}')
            #    return loss_train_history, loss_val_history, test_accuracy_history
    return loss_train_history, loss_val_history, test_accuracy_history

In [ ]:
# Example training call
loss_train_history, loss_val_history, test_accuracy_history = train(nn, train_images, train_labels, valid_images, valid_labels, test_images, test_labels, epochs=5)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss_train_history,label='Train Loss')
plt.plot(loss_val_history,label='Validation Loss')
plt.legend()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Step')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(test_accuracy_history)
plt.title('Test Set Accuracy')
plt.text(1950, test_accuracy_history[-1], str(test_accuracy_history[-1]), fontsize=8, ha='right')
plt.ylabel('Accuracy')
plt.xlabel('Step')
plt.show()